In [55]:
# imports
import pandas as pd

from copy import copy
from openpyxl import load_workbook
from openpyxl.cell import Cell
from openpyxl.formula.translate import Translator
from openpyxl.worksheet import worksheet
from pandas import DataFrame
from sqlalchemy import create_engine

# configuration
sqlDialect = 'mssql'
sqlDriver = 'pyodbc'
sqlConn = '@./Goal-Based-Database?'
driver = 'driver=ODBC+Driver+17+for+SQL+Server'

# inputs
sql = 'SELECT Cost, Cost FROM CashFlow'
template_name = 'Template.xlsx'
sh_name = 'CashFlow'
start_row = 2
output_name = 'output.xlsx'

# functions
def get_populated_worksheet(ws: worksheet, df: DataFrame, start_row: int) -> worksheet:
    cols = df.shape[1]
    rows = df.shape[0]
    for col in range(cols):
        reference = ws.cell(row=start_row, column=col + 1)
        for row in range(rows):
            cell = ws.cell(row=row+start_row+1, column=col+1)
            cell.value = get_cell_value(ws, df, cell, reference)
            cell = get_formatted_cell(cell,reference)
    ws.delete_rows(start_row)
    return ws

def get_formatted_cell(cell: Cell, reference: Cell) -> Cell:
    if reference.has_style:
        cell.font = copy(reference.font)
        cell.border = copy(reference.border)
        cell.fill = copy(reference.fill)
        cell.number_format = copy(reference.number_format)
        cell.protection = copy(reference.protection)
        cell.alignment = copy(reference.alignment)
    return cell

def get_cell_value(ws: worksheet, df: DataFrame, cell: Cell, reference: Cell):
    if reference.data_type == 'f':
        coordinate = ws.cell(row=cell.row - 1, column=cell.column).coordinate
        return Translator(reference.value, reference.coordinate).translate_formula(coordinate)
    else:
        df_row = cell.row - reference.row - 1
        return df.values[df_row,cell.column - 1]
    
# execute
engineConn = sqlDialect + '+' + sqlDriver + '://' + sqlConn + driver
engine = create_engine(engineConn)

df = pd.read_sql(sql, engine)
wb = load_workbook(filename = template_name)
ws = wb[sh_name]
ws = get_populated_worksheet(ws, df, start_row)
wb.save(output_name)